<a href="https://colab.research.google.com/github/michaelconsigli/Tesi/blob/main/Speech2RIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!git clone https://github.com/anton-jeran/Speech2RIR
%cd Speech2RIR/
!bash download_model.sh
!pip install torchmetrics

Cloning into 'Speech2RIR'...
remote: Enumerating objects: 237, done.
remote: Counting objects: 100% (237/237), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 237 (delta 55), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (237/237), 214.23 KiB | 6.69 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/Speech2RIR/Speech2RIR
Downloading...
From (original): https://drive.google.com/uc?id=1CcF1c9i76-MVPJ-PGoBwvaVtOUBPD57y
From (redirected): https://drive.google.com/uc?id=1CcF1c9i76-MVPJ-PGoBwvaVtOUBPD57y&confirm=t&uuid=15ac1d55-a026-4c0f-bc8e-a3b710c4eb04
To: /content/Speech2RIR/Speech2RIR/checkpoint-1040000steps.pkl
100% 3.38G/3.38G [01:07<00:00, 49.9MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1TrnXjR-vrCrub_RY6kdEBqA0D4hGiIp3
From (redirected): https://drive.google.com/uc?id=1TrnXjR-vrCrub_RY6kdEBqA0D4hGiIp3&confirm=t&uuid=c1d1cd39-a521-4557-b85f-1dc8b220174f
To: /content/Speech2RIR/Speech2RIR/checkpoint-1900000s


CARICO IL MODELLO:

- Importa le librerie — carica il codice di PyTorch e della rete (modello) specifico che userai.

- Istanzia il modello — crea una nuova rete (modello) usando il codice della classe Generator. Qui devi specificare le stesse dimensioni e parametri con cui la rete è stata addestrata, altrimenti i pesi non ci entreranno bene.

- Carica i pesi del modello — apre il checkpoint (file con dati salvati) e legge i pesi nella parte giusta del file (checkpoint['model']['generator']).

- Inserisci i pesi nel modello — qui è il punto critico: se le dimensioni del modello e dei pesi non combaciano esce errore.

- Metti il modello in modalità valutazione — prepara il modello per fare previsioni (usare il modello, non addestrarlo).

- Stampa messaggio di conferma.


I pesi verranno utilizzati nella funzione forward del generator, in particolare nell'encoder e nel decoder. Questi due usano pesi interni es.

*conv = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3*)

- Prima di caricare pesi, i valori dei filtri (kernel) sono casuali.

- Dopo conv.load_state_dict(...) con pesi addestrati, quei valori cambiano e diventano precisi per rilevare certe caratteristiche nell'input.

- Quando passi un input x a conv(x), il risultato riflette le trasformazioni con quei pesi.

In [9]:
# 1️⃣ Import delle librerie
import torch
from models.autoencoder.AudioDec import Generator

# 2️⃣ Crea il modello (con i parametri di default)
# Parametri dal file di configurazione YAML
model = Generator(
    input_channels=1,
    output_channels_rir=1,
    encode_channels=16,
    decode_channels=16,
    code_dim=128,
    codebook_num=64,
    codebook_size=8192,
    bias=True,
    combine_enc_ratios=[],
    combine_enc_strides=[],
    seperate_enc_ratios_rir=[2, 4, 8, 12, 16, 32],
    seperate_enc_strides_rir=[2, 2, 3, 5, 5, 5],
    rir_dec_ratios=[256, 128, 64, 32, 32, 32, 16],
    rir_dec_strides=[5, 5, 2, 2, 2, 1, 1],
    mode='causal',
    codec='audiodec',
    projector='conv1d',
    quantier='residual_vq',
)

# 3️⃣ Carica i pesi pre-addestrati
checkpoint_path = "exp/autoencoder/symAD_vctk_48000_hop300/checkpoint-1900000steps.pkl"
checkpoint = torch.load(checkpoint_path, map_location='cpu')

# Carica nello stato del modello solo la parte 'generator' dentro 'model'
model.load_state_dict(checkpoint['model']['generator'])

# 4️⃣ Metti il modello in modalità "valutazione"
model.eval()

print("✅ Modello caricato correttamente!")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Modello caricato correttamente!


- carico e preparo l'audio
- stimo la RIR
- salvo

In [14]:
# ============================================================
# 📘 CONFRONTO TRA RIR STIMATA (dal modello) E RIR REALE
# ============================================================
import torch
import torchaudio
import numpy as np
import torch.nn.functional as F
from torchmetrics.functional import pearson_corrcoef
import matplotlib.pyplot as plt
import sys
import os

# Aggiunge il percorso della directory 'losses' al sys.path
sys.path.append('/content/Speech2RIR/Speech2RIR/losses')

# ------------------------------------------------------------
# 1️⃣ CARICAMENTO AUDIO DI INGRESSO (speech riverberata)
# ------------------------------------------------------------
# L'audio di partenza è il segnale riverberato da cui il modello stima la RIR

audio_path = "/content/Tests/Test 1/impulseresponseheslingtonchurch-006_sing.mp3"
waveform, sr = torchaudio.load(audio_path)

# 🔸 Porta tutto a 48 kHz (il modello è addestrato così)
if sr != 48000:
    waveform = torchaudio.functional.resample(waveform, sr, 48000)
    sr = 48000

# 🔸 Converte in mono (serve 1 solo canale)
if waveform.shape[0] > 1:
    waveform = torch.mean(waveform, dim=0, keepdim=True)

# 🔸 Aggiunge dimensione batch (B, C, T)
waveform = waveform.unsqueeze(0)

# ------------------------------------------------------------
# 2️⃣ STIMA DELLA RIR TRAMITE MODELLO (già caricato come “model”)
# ------------------------------------------------------------

model.eval()
with torch.no_grad():
    estimated_rir = model(waveform)  # output: (B, 1, T)

print("✅ RIR stimata dal modello.")
print("Shape stimata:", estimated_rir.shape)

# ------------------------------------------------------------
# 3️⃣ CARICAMENTO DELLA RIR REALE ASSOCIATA
# ------------------------------------------------------------
rir_real_path = "/content/Tests/Test 1/impulseresponseheslingtonchurch-006.wav"
rir_real, sr_rir = torchaudio.load(rir_real_path)

if sr_rir != 48000:
    rir_real = torchaudio.functional.resample(rir_real, sr_rir, 48000)

rir_real = rir_real.unsqueeze(0) if rir_real.ndim == 1 else rir_real
rir_real = torch.mean(rir_real, dim=0, keepdim=True)  # mono

print("✅ RIR reale caricata.")
print("Shape reale:", rir_real.shape)

# ------------------------------------------------------------
# 4️⃣ NORMALIZZAZIONE AMPIEZZA (per confronto forma, non livello)
# ------------------------------------------------------------
estimated_rir = estimated_rir / torch.max(torch.abs(estimated_rir))
rir_real = rir_real / torch.max(torch.abs(rir_real))

# ------------------------------------------------------------
# 5️⃣ ALLINEAMENTO TEMPORALE (cross-correlation)
# ------------------------------------------------------------
# Il modello può introdurre un piccolo ritardo → lo stimiamo e correggiamo
corr = F.conv1d(rir_real, torch.flip(estimated_rir, dims=[-1]), padding=rir_real.shape[-1]//2)
shift = torch.argmax(corr)
shift_amount = int(shift - rir_real.shape[-1]//2)
rir_real_aligned = torch.roll(rir_real, -shift_amount, dims=-1)

print(f"Allineamento temporale corretto: shift di {shift_amount} campioni.")

# ------------------------------------------------------------
# 6️⃣ TAGLIO A LUNGHEZZA MINIMA COMUNE
# ------------------------------------------------------------
min_len = min(rir_real_aligned.shape[-1], estimated_rir.shape[-1])
rir_real_aligned = rir_real_aligned[..., :min_len]
estimated_rir = estimated_rir[..., :min_len]

# ------------------------------------------------------------
# 7️⃣ CALCOLO DELLE METRICHE DI CONFRONTO
# ------------------------------------------------------------

# 🔸 MSE (Mean Squared Error)
mse = torch.mean((rir_real_aligned - estimated_rir) ** 2).item()

# 🔸 Correlazione di Pearson (PCC)
corr_coeff = pearson_corrcoef(rir_real_aligned.flatten(), estimated_rir.flatten()).item()

# 🔸 Differenza Spettrale (STFT distance)
spec_real = torch.stft(rir_real_aligned.squeeze(), n_fft=1024, hop_length=256, return_complex=True)
spec_est = torch.stft(estimated_rir.squeeze(), n_fft=1024, hop_length=256, return_complex=True)
stft_distance = torch.mean(torch.abs(torch.log1p(torch.abs(spec_real)) - torch.log1p(torch.abs(spec_est)))).item()
print("\n📊 RISULTATI DEL CONFRONTO")
print(f"• MSE: {mse:.6f}")
print(f"• Correlazione (PCC): {corr_coeff:.4f}")
print(f"• Distanza spettrale (STFT): {stft_distance:.6f}")

# ------------------------------------------------------------
# 8️⃣ VISUALIZZAZIONE GRAFICA DELLE DUE RIR
# ------------------------------------------------------------
#time = np.linspace(0, min_len / 48000, min_len)

#plt.figure(figsize=(12, 5))
##plt.plot(time, rir_real_aligned.squeeze().cpu().numpy(), label='RIR Reale', alpha=0.8)
#plt.plot(time, estimated_rir.squeeze().cpu().numpy(), label='RIR Stimata', alpha=0.7)
#plt.xlabel("Tempo [s]")
#plt.ylabel("Ampiezza normalizzata")
#plt.title("Confronto tra RIR Reale e RIR Stimata")
#plt.legend()
#plt.grid()
#plt.show()

✅ RIR stimata dal modello.
Shape stimata: torch.Size([1, 1, 79200])
✅ RIR reale caricata.
Shape reale: torch.Size([1, 144000])
Allineamento temporale corretto: shift di -58689 campioni.

📊 RISULTATI DEL CONFRONTO
• MSE: 0.004858
• Correlazione (PCC): -0.0000
• Distanza spettrale (STFT): 0.442082
